# Machine Learning Models Showcase

## Load data from database

In [1]:
# imports
import pandas as pd
import sklearn as skl
import psycopg2
import config as c

# import for multiple output per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# connection to database
connection = psycopg2.connect(
    host = c.host,
    port = c.port,
    user = c.user,
    password = c.password,
    database = c.database
    )
cursor=connection.cursor()

In [3]:
data_sql = """
SELECT *
FROM comprehensive_dataset;
"""

#  load in tables as dataframes
data_df = pd.read_sql(data_sql, con=connection)
data_df.head()

,GUID,Name,HallOfFameStatus,HallofFameClass,YearDrafted,TO_YEAR,Years_Played,HOF_Elgibility_Year,Pick,Team,...,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,EFF
0,MIKENILES1980,Mike Niles,Not Inducted,NaN,1980,1980,1,1985,NaN,None,...,0.8,45.9,0.6,0.7,1.3,0.3,0.2,0.0,0.6,1.4
1,WAYNEROBINSON1980,Wayne Robinson,Not Inducted,NaN,1980,1980,1,1985,31.0,LAL,...,3.0,72.9,1.4,2.2,3.6,1.4,0.6,0.3,1.8,7.8
2,BILLYREID1980,Billy Reid,Not Inducted,NaN,1980,1980,1,1985,182.0,GSW,...,0.7,56.4,0.5,0.6,1.0,1.2,0.6,0.1,1.3,2.8
3,ALEXBRADLEY1981,Alex Bradley,Not Inducted,NaN,1981,1981,1,1986,86.0,NYK,...,1.2,60.4,0.8,0.9,1.7,0.3,0.3,0.1,0.7,3.4
4,GARRYWITTS1981,Garry Witts,Not Inducted,NaN,1981,1981,1,1986,103.0,WSB,...,0.9,82.5,0.6,0.7,1.3,0.8,0.4,0.1,0.8,3.8


## Preprocessing 

In [4]:
# get_dummies to encode HallofFameClass column
data_df = pd.get_dummies(data_df, prefix='HOF', columns=['HallOfFameStatus'])
data_df.drop(['HOF_Not Inducted'], axis=1, inplace=True) # remove extra get_dummies column, drop_first no ideal

# drop columns that will likely error/confuse the ML models
data_df.drop(['GUID', 'Name', 'Team', 'College', # dropping object columns 
              'HallofFameClass', 'YearDrafted', 'TO_YEAR', 'HOF_Elgibility_Year', 'GP', 'MIN' # non performance stats
                ], axis=1, inplace=True)

# fill NaNs with 0
data_df['Pick'] = data_df['Pick'].fillna(0)

data_df.head()
print(data_df.shape)

,Years_Played,Pick,PTS,FGM,FGA,FG%,3P_Made,3PA,3P%,FTM,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,EFF,HOF_Hall of Fame Member
0,1,0.0,2.6,1.1,3.1,34.8,0.0,0.1,50.0,0.4,...,45.9,0.6,0.7,1.3,0.3,0.2,0.0,0.6,1.4,0
1,1,31.0,7.9,2.9,6.3,46.0,0.0,0.1,0.0,2.2,...,72.9,1.4,2.2,3.6,1.4,0.6,0.3,1.8,7.8,0
2,1,182.0,3.2,1.4,3.1,45.4,0.0,0.1,0.0,0.4,...,56.4,0.5,0.6,1.0,1.2,0.6,0.1,1.3,2.8,0
3,1,86.0,3.5,1.4,2.6,52.4,0.0,0.0,0.0,0.7,...,60.4,0.8,0.9,1.7,0.3,0.3,0.1,0.7,3.4,0
4,1,103.0,2.9,1.1,1.8,58.3,0.0,0.0,50.0,0.7,...,82.5,0.6,0.7,1.3,0.8,0.4,0.1,0.8,3.8,0


(1217, 21)


## Machine Learning Models

### LogReg w/ Oversampling

In [5]:
# ML imports
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced
import tensorflow as tf

In [6]:
# create our features
X = data_df.drop(['HOF_Hall of Fame Member'], axis=1)

# create our target
y = data_df['HOF_Hall of Fame Member']

In [7]:
# normal train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)
Counter(y_train)

Counter({1: 28, 0: 884})

#### RandomOverSampler

In [8]:
# import RandomOverSampler
from imblearn.over_sampling import RandomOverSampler

In [9]:
# implement random oversampling
X_resampled, y_resampled = RandomOverSampler(random_state=2).fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({1: 884, 0: 884})

In [10]:
# logistic regression using random oversampled data
model = LogisticRegression(solver='liblinear', random_state=2)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=2, solver='liblinear')

In [11]:
# generate predictions on test set
y_pred_test = model.predict(X_test)

# calculated the accuracy scores
test_acc_score = balanced_accuracy_score(y_test, y_pred_test)
train_acc_score = model.score(X_train, y_train)

In [12]:
# define confusion matrix
cm = confusion_matrix(y_test, y_pred_test)

# create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Not HOF Member", "HOF Member"], 
    columns=["Predicted Not HOF Member", "Predicted HOF Member"])

In [13]:
# RandomOverSampler results
print("CONFUSION MATRIX")
display(cm_df)
print("ACCURACY SCORES\n")
print(f'Training Accuracy: {train_acc_score}\n')
print(f'Testing Accuracy: {test_acc_score}\n')
print("CLASSIFICATION REPORT\n")
print(classification_report_imbalanced(y_test, y_pred_test))

CONFUSION MATRIX


,Predicted Not HOF Member,Predicted HOF Member
Not HOF Member,252,39
HOF Member,3,11


ACCURACY SCORES

Training Accuracy: 0.8892543859649122

Testing Accuracy: 0.8258468335787923

CLASSIFICATION REPORT

                   pre       rec       spe        f1       geo       iba       sup

          0       0.99      0.87      0.79      0.92      0.82      0.69       291
          1       0.22      0.79      0.87      0.34      0.82      0.67        14

avg / total       0.95      0.86      0.79      0.90      0.82      0.69       305



#### SMOTE

In [14]:
# import SMOTE
from imblearn.over_sampling import SMOTE

In [15]:
# implement SMOTE resampling
X_resampled, y_resampled = SMOTE(random_state=2, sampling_strategy='auto').fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({1: 884, 0: 884})

In [16]:
# logistic regression using SMOTE data
model = LogisticRegression(solver='liblinear', random_state=2)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=2, solver='liblinear')

In [17]:
# generate predictions on test set
y_pred_test = model.predict(X_test)

# calculated the accuracy scores
test_acc_score = balanced_accuracy_score(y_test, y_pred_test)
train_acc_score = model.score(X_train, y_train)

In [18]:
# define confusion matrix
cm = confusion_matrix(y_test, y_pred_test)

# create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Not HOF Member", "HOF Member"], 
    columns=["Predicted Not HOF Member", "Predicted HOF Member"])

In [19]:
# SMOTE results
print("CONFUSION MATRIX")
display(cm_df)
print("ACCURACY SCORES\n")
print(f'Training Accuracy: {train_acc_score}\n')
print(f'Testing Accuracy: {test_acc_score}\n')
print("CLASSIFICATION REPORT\n")
print(classification_report_imbalanced(y_test, y_pred_test))

CONFUSION MATRIX


,Predicted Not HOF Member,Predicted HOF Member
Not HOF Member,256,35
HOF Member,1,13


ACCURACY SCORES

Training Accuracy: 0.8914473684210527

Testing Accuracy: 0.9041482572410408

CLASSIFICATION REPORT

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.88      0.93      0.93      0.90      0.81       291
          1       0.27      0.93      0.88      0.42      0.90      0.82        14

avg / total       0.96      0.88      0.93      0.91      0.90      0.81       305



#### SVM SMOTE

In [20]:
# import SVMSMOTE
from imblearn.over_sampling import SVMSMOTE

In [21]:
## implement SVM SMOTE resampling
X_resampled, y_resampled = SVMSMOTE(random_state=2, sampling_strategy='auto').fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({1: 431, 0: 884})

In [22]:
# logistic regression using SVM SMOTE data
model = LogisticRegression(solver='liblinear', random_state=2)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=2, solver='liblinear')

In [23]:
# generate predictions on test set
y_pred_test = model.predict(X_test)

# calculated the accuracy scores
test_acc_score = balanced_accuracy_score(y_test, y_pred_test)
train_acc_score = model.score(X_train, y_train)

In [24]:
# define confusion matrix
cm = confusion_matrix(y_test, y_pred_test)

# create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Not HOF Member", "HOF Member"], 
    columns=["Predicted Not HOF Member", "Predicted HOF Member"])

In [25]:
# SVMSMOTE results
print("CONFUSION MATRIX")
display(cm_df)
print("ACCURACY SCORES\n")
print(f'Training Accuracy: {train_acc_score}\n')
print(f'Testing Accuracy: {test_acc_score}\n')
print("CLASSIFICATION REPORT\n")
print(classification_report_imbalanced(y_test, y_pred_test))

CONFUSION MATRIX


,Predicted Not HOF Member,Predicted HOF Member
Not HOF Member,273,18
HOF Member,2,12


ACCURACY SCORES

Training Accuracy: 0.9418859649122807

Testing Accuracy: 0.8976435935198821

CLASSIFICATION REPORT

                   pre       rec       spe        f1       geo       iba       sup

          0       0.99      0.94      0.86      0.96      0.90      0.81       291
          1       0.40      0.86      0.94      0.55      0.90      0.80        14

avg / total       0.97      0.93      0.86      0.95      0.90      0.81       305



In [ ]:
# Classification Report Explained
# All these are metrics for measuring performance of imbalanced classes.

# pre stands for precision, ratio of TP to sum of TP and FP
# rec stands for recall/sensitivity, ratio of TP to sum of TP and FN
# spe stands for specificity, ratio of how many were correctly classified
# f1 stands for f1 measure,  harmonic mean of pre and rec
# geo stands for geometric mean, square root of the product of rec and spe
# iba stands for index balanced accuracy
# sup stands for support, the number of points per class

## Decision Tree and Neural Net

### Scale the data

In [26]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Decision Tree Model

In [27]:
# DT imports
from path import Path
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [28]:
# create the decision tree classifier instance.
model = tree.DecisionTreeClassifier()

# fit the model.
model = model.fit(X_train_scaled, y_train)

In [29]:
# DT predictions
predictions = model.predict(X_test_scaled)

# calcuate the accuracy score.
test_acc_score = accuracy_score(y_test, predictions)
train_acc_score = model.score(X_train, y_train)

In [30]:
# define confusion matrix
cm = confusion_matrix(y_test, y_pred_test)

# create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Not HOF Member", "HOF Member"], 
    columns=["Predicted Not HOF Member", "Predicted HOF Member"])

In [31]:
# DT results
print("CONFUSION MATRIX")
display(cm_df)
print("ACCURACY SCORES\n")
print(f'Training Accuracy: {train_acc_score}\n')
print(f'Testing Accuracy: {test_acc_score}\n')
print("CLASSIFICATION REPORT\n")
print(classification_report_imbalanced(y_test, y_pred_test))

CONFUSION MATRIX


,Predicted Not HOF Member,Predicted HOF Member
Not HOF Member,273,18
HOF Member,2,12


ACCURACY SCORES

Training Accuracy: 0.7861842105263158

Testing Accuracy: 0.940983606557377

CLASSIFICATION REPORT

                   pre       rec       spe        f1       geo       iba       sup

          0       0.99      0.94      0.86      0.96      0.90      0.81       291
          1       0.40      0.86      0.94      0.55      0.90      0.80        14

avg / total       0.97      0.93      0.86      0.95      0.90      0.81       305



## Neural Net

In [32]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [33]:
# Neural Net model
# Define the basic neural network model
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=10, activation="relu", input_dim=20))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
29/29 [==============================] - 0s 641us/step - loss: 0.5927 - accuracy: 0.6656
Epoch 2/100
29/29 [==============================] - 0s 677us/step - loss: 0.4837 - accuracy: 0.8289
Epoch 3/100
29/29 [==============================] - 0s 641us/step - loss: 0.3978 - accuracy: 0.9353
Epoch 4/100
29/29 [==============================] - 0s 641us/step - loss: 0.3292 - accuracy: 0.9594
Epoch 5/100
29/29 [==============================] - 0s 677us/step - loss: 0.2783 - accuracy: 0.9638
Epoch 6/100
29/29 [==============================] - 0s 677us/step - loss: 0.2410 - accuracy: 0.9660
Epoch 7/100
29/29 [==============================] - 0s 712us/step - loss: 0.2154 - accuracy: 0.9671
Epoch 8/100
29/29 [==============================] - 0s 641us/step - loss: 0.1966 - accuracy: 0.9693
Epoch 9/100
29/29 [==============================] - 0s 677us/step - loss: 0.1825 - accuracy: 0.9693
Epoch 10/100
29/29 [==============================] - 0s 677us/step - loss: 0.1714 - accura

29/29 [==============================] - 0s 641us/step - loss: 0.0639 - accuracy: 0.9759
Epoch 82/100
29/29 [==============================] - 0s 606us/step - loss: 0.0633 - accuracy: 0.9759
Epoch 83/100
29/29 [==============================] - 0s 606us/step - loss: 0.0635 - accuracy: 0.9770
Epoch 84/100
29/29 [==============================] - 0s 606us/step - loss: 0.0637 - accuracy: 0.9759
Epoch 85/100
29/29 [==============================] - 0s 606us/step - loss: 0.0623 - accuracy: 0.9770
Epoch 86/100
29/29 [==============================] - 0s 606us/step - loss: 0.0622 - accuracy: 0.9781
Epoch 87/100
29/29 [==============================] - 0s 641us/step - loss: 0.0622 - accuracy: 0.9781
Epoch 88/100
29/29 [==============================] - 0s 641us/step - loss: 0.0616 - accuracy: 0.9770
Epoch 89/100
29/29 [==============================] - 0s 641us/step - loss: 0.0617 - accuracy: 0.9781
Epoch 90/100
29/29 [==============================] - 0s 606us/step - loss: 0.0611 - accuracy: 

## Deep Learning

In [34]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
29/29 [==============================] - 0s 712us/step - loss: 0.7239 - accuracy: 0.5428
Epoch 2/100
29/29 [==============================] - 0s 784us/step - loss: 0.5189 - accuracy: 0.7895
Epoch 3/100
29/29 [==============================] - 0s 890us/step - loss: 0.3865 - accuracy: 0.8980
Epoch 4/100
29/29 [==============================] - 0s 748us/step - loss: 0.3048 - accuracy: 0.9342
Epoch 5/100
29/29 [==============================] - 0s 748us/step - loss: 0.2523 - accuracy: 0.9550
Epoch 6/100
29/29 [==============================] - 0s 819us/step - loss: 0.2185 - accuracy: 0.9638
Epoch 7/100
29/29 [==============================] - 0s 748us/step - loss: 0.1934 - accuracy: 0.9671
Epoch 8/100
29/29 [==============================] - 0s 748us/step - loss: 0.1745 - accuracy: 0.9682
Epoch 9/100
29/29 [==============================] - 0s 784us/step - loss: 0.1587 - accuracy: 0.9693
Epoch 10/100
29/29 [==============================] - 0s 748us/step - loss: 0.1462 - accura

29/29 [==============================] - 0s 748us/step - loss: 0.0625 - accuracy: 0.9737
Epoch 82/100
29/29 [==============================] - 0s 748us/step - loss: 0.0616 - accuracy: 0.9715
Epoch 83/100
29/29 [==============================] - 0s 641us/step - loss: 0.0615 - accuracy: 0.9715
Epoch 84/100
29/29 [==============================] - 0s 677us/step - loss: 0.0612 - accuracy: 0.9726
Epoch 85/100
29/29 [==============================] - 0s 641us/step - loss: 0.0613 - accuracy: 0.9726
Epoch 86/100
29/29 [==============================] - 0s 641us/step - loss: 0.0601 - accuracy: 0.9726
Epoch 87/100
29/29 [==============================] - 0s 641us/step - loss: 0.0597 - accuracy: 0.9726
Epoch 88/100
29/29 [==============================] - 0s 677us/step - loss: 0.0599 - accuracy: 0.9759
Epoch 89/100
29/29 [==============================] - 0s 677us/step - loss: 0.0587 - accuracy: 0.9737
Epoch 90/100
29/29 [==============================] - 0s 677us/step - loss: 0.0587 - accuracy: 